In [1]:
import pandas as pd
from ast import literal_eval
from nltk.tokenize import sent_tokenize, word_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic
import spacy

# Load Datasets

We load the preprocessed datasets. Make sure to run the previous notebooks first, so the files are present.

## media articles

In [2]:
df_media = pd.read_csv('../data/cleantech-media.csv', converters={"token_content": literal_eval}, parse_dates=['date'])
df_media.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9588 entries, 0 to 9587
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          9588 non-null   object        
 1   date           9588 non-null   datetime64[ns]
 2   author         31 non-null     object        
 3   content        9588 non-null   object        
 4   domain         9588 non-null   object        
 5   url            9588 non-null   object        
 6   token_content  9588 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 524.5+ KB


In [3]:
df_media.head(5)

,title,date,author,content,domain,url,token_content
0,Qatar to Slash Emissions as LNG Expansion Adva...,2021-01-13,NaN,Qatar Petroleum ( QP) is targeting aggressive ...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"[qatar, petroleum, qp, target, aggress, cut, g..."
1,India Launches Its First 700 MW PHWR,2021-01-15,NaN,Nuclear Power Corp. of India Ltd. ( NPCIL) syn...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"[nuclear, power, corp, india, ltd, npcil, sync..."
2,New Chapter for US-China Energy Trade,2021-01-20,NaN,New US President Joe Biden took office this we...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"[new, us, presid, joe, biden, take, offic, wee..."
3,Japan: Slow Restarts Cast Doubt on 2030 Energy...,2021-01-22,NaN,The slow pace of Japanese reactor restarts con...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"[slow, pace, japanes, reactor, restart, contin..."
4,NYC Pension Funds to Divest Fossil Fuel Shares,2021-01-25,NaN,Two of New York City's largest pension funds s...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"[two, new, york, citi, larg, pension, fund, sa..."


## patents

In [4]:
df_patents = pd.read_csv('../data/google_patents.csv', converters={"token_content": literal_eval}, parse_dates=['publication_date'])
df_patents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13297 entries, 0 to 13296
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   publication_number  13297 non-null  object        
 1   application_number  13297 non-null  object        
 2   country_code        13297 non-null  object        
 3   publication_date    13297 non-null  datetime64[ns]
 4   inventor            13297 non-null  object        
 5   title               13297 non-null  object        
 6   title_lang          13297 non-null  object        
 7   abstract            13297 non-null  object        
 8   abstract_lang       13297 non-null  object        
 9   token_content       13297 non-null  object        
dtypes: datetime64[ns](1), object(9)
memory usage: 1.0+ MB


In [40]:
# TODO: remove this as soon as translated
df_patents = df_patents.loc[df_patents.abstract_lang == 'en', :]
df_patents.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6680 entries, 0 to 13295
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   publication_number  6680 non-null   object        
 1   application_number  6680 non-null   object        
 2   country_code        6680 non-null   object        
 3   publication_date    6680 non-null   datetime64[ns]
 4   inventor            6680 non-null   object        
 5   title               6680 non-null   object        
 6   title_lang          6680 non-null   object        
 7   abstract            6680 non-null   object        
 8   abstract_lang       6680 non-null   object        
 9   token_content       6680 non-null   object        
dtypes: datetime64[ns](1), object(9)
memory usage: 574.1+ KB


In [42]:
df_patents.head()

,publication_number,application_number,country_code,publication_date,inventor,title,title_lang,abstract,abstract_lang,token_content
0,US-2022239235-A1,US-202217717397-A,US,2022-07-28,[],Adaptable DC-AC Inverter Drive System and Oper...,en,Disclosed is an adaptable DC-AC inverter syste...,en,"[disclos, adapt, dc, ac, invert, system, oper,..."
1,US-2022239251-A1,US-202217580956-A,US,2022-07-28,[],System for providing the energy from a single ...,en,"In accordance with an example embodiment, a so...",en,"[accord, exampl, embodi, solar, energi, system..."
3,US-11396827-B2,US-202117606042-A,US,2022-07-26,[],Control method for optimizing solar-to-power e...,en,A control method for optimizing a solar-to-pow...,en,"[control, method, optim, solar, power, effici,..."
6,CN-114777546-A,CN-202210702520-A,CN,2022-07-22,[],Cold and hot medium energy storage hot water a...,en,The invention relates to the technical field o...,en,"[invent, relat, technic, field, energi, storag..."
11,CN-114771214-A,CN-202210621034-A,CN,2022-07-22,[],氢能源汽车空调滤清器,zh,The invention discloses a hydrogen energy auto...,en,"[invent, disclos, hydrogen, energi, automobil,..."


# Modeling with BERTopic

## Preprocessing

### split up large chunks of text

In [38]:
text_corpus_media = df_media.token_content.apply(lambda x: ' '.join(x))

sentences_media = [sent_tokenize(doc) for doc in text_corpus_media]
sentences_media = [sentence for doc in sentences_media for sentence in doc]

sentences_media[0:2]

['qatar petroleum qp target aggress cut greenhous ga emiss prepar launch phase 2 plan 48 million ton per year lng expans late sustain report publish wednesday qp say goal includ reduc emiss intens qatar lng facil 25 upstream facil least 15 compani also aim reduc ga flare intens across upstream facil 75 rais carbon captur storag ambit 5 million ton yr 7 million ton yr 2027 2 2 million ton yr carbon captur goal come 32 million ton yr phase 1 lng expans also know north field east project 1 1 million ton yr come phase 2 known north field south project rais qatar lng capac 16 million ton yr qatar current lng product capac around 78 million ton yr eye phase expans 126 million ton yr qp say abl elimin routin ga flare 2030 methan emiss limit set methan intens target 0 2 across facil 2025 compani also plan build 1 6 gigawatt solar energi capac 2025 half come siraj solar power project next year eif jan 22 20 month littl news phase 2 qatar massiv lng expans mcdermott intern say last week award fr

In [39]:
len(sentences_media)

9588

In [43]:
text_corpus_patents = df_patents.token_content.apply(lambda x: ' '.join(x))

sentences_patents = [sent_tokenize(doc) for doc in text_corpus_patents]
sentences_patents = [sentence for doc in sentences_patents for sentence in doc]

sentences_patents[0:2]

['disclos adapt dc ac invert system oper system includ multipl dc input sourc input provid stabl oper variou condit dc input sourc may add system remov system without impact function system disclos system suit solar energi harvest grid connect grid mode oper',
 'accord exampl embodi solar energi system compris solar energi structur compris photovolta solar panel contigu cover area first invert configur receiv power first string photovolta solar panel wherein first invert configur provid power receiv first invert first meter second invert configur receiv power second string photovolta solar panel wherein second invert configur provid power receiv second invert second meter']

In [44]:
len(sentences_patents)

6680

### Limit training samples

Depending on where you run this you might want to reduce training samples

In [69]:
# Limit samples to constant MAX_SAMPLES
MAX_SAMPLES = 10_000

training_sentences_media = sentences_media[0:MAX_SAMPLES]
training_sentences_patents = sentences_patents[0:MAX_SAMPLES]

In [55]:
(len(training_sentences_media), len(training_sentences_patents))

(9588, 6680)

### Calculate Embeddings

We pre-calculate the embeddings to feed them into BERT. This saves us time later, as we do not have to recalculate this step every time.

In [71]:
embedding_model_media = SentenceTransformer("all-MiniLM-L6-v2")

embeddings_media = embedding_model_media.encode(training_sentences_media, show_progress_bar=True)

Batches:   0%|          | 0/300 [00:00<?, ?it/s]

In [57]:
embeddings_media.shape

(9588, 384)

In [70]:
embedding_model_patents = SentenceTransformer("all-MiniLM-L6-v2")

embeddings_patents = embedding_model_patents.encode(training_sentences_patents, show_progress_bar=True)

Batches:   0%|          | 0/209 [00:00<?, ?it/s]

In [59]:
embeddings_patents.shape

(6680, 384)

### Dimensionality reduction model (UMAP)

In [60]:
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

### Clustering (HDBSCAN)

In [85]:
hdbscan_model = HDBSCAN(min_cluster_size=25, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

### Vectorizer

In [63]:
vectorizer_model = CountVectorizer()

### Representation models

In [75]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, PartOfSpeech

# KeyBERT
keybert_model = KeyBERTInspired()

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    "MMR": mmr_model,
}

## Train 

In [86]:
topic_model_media = BERTopic(

  # Pipeline models
  embedding_model=embedding_model_media,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

topics_media, probs_media = topic_model_media.fit_transform(training_sentences_media, embeddings_media)

2024-04-26 17:06:00,692 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-26 17:06:12,950 - BERTopic - Dimensionality - Completed ✓
2024-04-26 17:06:12,951 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-26 17:06:13,162 - BERTopic - Cluster - Completed ✓
2024-04-26 17:06:13,165 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-26 17:06:19,681 - BERTopic - Representation - Completed ✓


In [87]:
topic_model_media.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,1685,-1_solar_energi_pv_power,"[solar, energi, pv, power, storag, project, sy...","[solar, pv, industri, panel, technolog, electr...","[solar, energi, pv, power, storag, project, sy...",[solar electr 16 2022 rel modest increas due l...
1,0,4938,0_energi_ga_geotherm_develop,"[energi, ga, geotherm, develop, hydrogen, say,...","[environment, industri, fuel, sector, oil, car...","[energi, ga, geotherm, develop, hydrogen, say,...",[click allow agre store cooki devic enhanc sit...
2,1,780,1_vehicl_tesla_ev_electr,"[vehicl, tesla, ev, electr, charg, car, cleant...","[tesla, ev, vehicl, ford, car, electr, volkswa...","[vehicl, tesla, ev, electr, charg, car, cleant...",[plugin electr vehicl fulli electr vehicl well...
3,2,617,2_solar_energi_project_renew,"[solar, energi, project, renew, power, commun,...","[solar, sector, energi, industri, build, proje...","[solar, energi, project, renew, power, commun,...",[consum energi princip subsidiari cm energi ag...
4,3,278,3_power_grid_system_batteri,"[power, grid, system, batteri, energi, storag,...","[microgrid, solar, pv, grid, voltag, panel, el...","[power, grid, system, batteri, energi, storag,...",[follow top solar invert product 2021 take loo...
5,4,226,4_batteri_lithium_ion_materi,"[batteri, lithium, ion, materi, energi, techno...","[lithium, batteri, electr, ev, vehicl, manufac...","[batteri, lithium, ion, materi, energi, techno...",[low carbon energi revolut becom unstopp jugge...
6,5,182,5_solar_manufactur_modul_pv,"[solar, manufactur, modul, pv, us, panel, year...","[solar, industri, tariff, panel, sector, pv, m...","[solar, manufactur, modul, pv, us, panel, year...",[solar energi industri associ seia wood macken...
7,6,152,6_dn_media_group_recharg,"[dn, media, group, recharg, read, privaci, dat...","[sector, industri, privaci, data, polici, farm...","[dn, media, group, recharg, read, privaci, dat...",[global offshor wind player bluefloat energi l...
8,7,119,7_cell_perovskit_effici_solar,"[cell, perovskit, effici, solar, pv, data, mod...","[photovolta, solar, pv, cell, industri, silico...","[cell, perovskit, effici, solar, pv, data, mod...",[result confirm germani institut solar energi ...
9,8,88,8_wind_power_renew_solar,"[wind, power, renew, solar, energi, coal, gene...","[coal, solar, demand, pv, electr, fuel, power,...","[wind, power, renew, solar, energi, coal, gene...",[accord new data u feder energi regulatori com...


In [88]:
topic_model_patents = BERTopic(

  # Pipeline models
  embedding_model=embedding_model_patents,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

topics_patents, probs_patents = topic_model_patents.fit_transform(training_sentences_patents, embeddings_patents)

2024-04-26 17:06:20,222 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-26 17:06:29,020 - BERTopic - Dimensionality - Completed ✓
2024-04-26 17:06:29,021 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-26 17:06:29,161 - BERTopic - Cluster - Completed ✓
2024-04-26 17:06:29,162 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-26 17:06:31,931 - BERTopic - Representation - Completed ✓


In [89]:
topic_model_patents.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,2214,-1_solar_energi_panel_connect,"[solar, energi, panel, connect, arrang, devic,...","[photovolta, solar, panel, electr, system, mot...","[solar, energi, panel, connect, arrang, devic,...",[util model disclos electr storag devic compri...
1,0,888,0_heat_water_energi_system,"[heat, water, energi, system, solar, temperatu...","[heater, solar, photovolta, thermal, heat, pum...","[heat, water, energi, system, solar, temperatu...",[invent disclos build energi supplement system...
2,1,701,1_water_plate_connect_hydropow,"[water, plate, connect, hydropow, fix, conserv...","[pipe, clamp, plate, flow, hydropow, drainag, ...","[water, plate, connect, hydropow, fix, conserv...",[util model relat technic field water conserv ...
3,2,441,2_wind_gener_power_blade,"[wind, gener, power, blade, energi, rotor, rot...","[wind, rotor, vehicl, wheel, grid, speed, rota...","[wind, gener, power, blade, energi, rotor, rot...",[invent disclos doubl wind wheel wind turbin g...
4,3,315,3_water_solar_plant_energi,"[water, solar, plant, energi, util, devic, pip...","[panel, environ, inlet, solar, system, model, ...","[water, solar, plant, energi, util, devic, pip...",[util model disclos multi theme environ friend...
5,4,303,4_lamp_light_street_solar,"[lamp, light, street, solar, connect, panel, r...","[lamp, solar, panel, lampshad, light, led, pol...","[lamp, light, street, solar, connect, panel, r...",[util model relat street lamp particular 5g co...
6,5,293,5_hydrogen_system_fuel_energi,"[hydrogen, system, fuel, energi, storag, produ...","[hydrogen, fuel, system, inlet, tank, pipe, pr...","[hydrogen, system, fuel, energi, storag, produ...",[util model relat heat control system hydrogen...
7,6,266,6_power_modul_voltag_circuit,"[power, modul, voltag, circuit, charg, control...","[photovolta, solar, voltag, panel, circuit, po...","[power, modul, voltag, circuit, charg, control...",[util model provid power suppli devic uninterr...
8,7,262,7_hydropow_station_data_method,"[hydropow, station, data, method, water, calcu...","[hydropow, reservoir, hydroelectr, system, sta...","[hydropow, station, data, method, water, calcu...",[invent relat technic field hydropow gener pro...
9,8,190,8_solar_panel_connect_plate,"[solar, panel, connect, plate, fix, frame, rod...","[panel, solar, frame, clamp, pivot, bracket, m...","[solar, panel, connect, plate, fix, frame, rod...",[util model suitabl technic field solar panel ...


## Refine


### Experimental: get topic names with local LLM

In [90]:
import json
import requests

In [91]:
def get_ollama_topic(representative_docs, keywords, model='llama2'):

    # URL of the API
    url = "http://localhost:11434/api/generate"

    topic_prompt = f"""
    I have a topic that contains the following documents:
    {representative_docs}
    The topic is described by the following keywords: {keywords}
    
    Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
    topic: <topic label>
    """
    
    # Data to be sent
    data = {
        "model": model,
        "prompt": topic_prompt,
        "stream": False
    }
    
    # Send POST request with data
    response = requests.post(url, json=data)
    
    # Print response
    response_text = json.loads(response.text).get('response', None)
    
    return response_text.strip().replace('Topic: ', '').splitlines()[0]

In [92]:
# get topic names from LLM and prepare dict
topic_dict_media = {x['Topic']: get_ollama_topic(x['Representative_Docs'], x['Representation']) for x in topic_model_media.get_topic_info().to_dict(orient='records')}
topic_dict_media[-1] = 'Outlier Topic'
topic_dict_media

{-1: 'Outlier Topic',
 0: 'Renewable Energi Developments',
 1: 'topic: Tesla ev sales',
 2: 'Solar Energi Projects',
 3: 'Grid-tied batteri storage',
 4: 'Lithium ion batteri',
 5: 'Solar Manufacturing',
 6: 'Offshore Wind Energy',
 7: 'Perovskit Solar Cell Efficiency',
 8: 'Renewable Power Growth',
 9: 'Rig Count Week Ga Oil',
 10: 'European Solar Landscape',
 11: 'Nuclear Power',
 12: 'topic: South African solar power',
 13: 'Renewable Energy Projects in Australia',
 14: 'Agrivolta solar farming',
 15: 'topic: Electric Bikes',
 16: 'Solar Job Growth',
 17: 'Solar Panel Modul Bifaci'}

In [93]:
# get topic names from LLM and prepare dict
topic_dict_patents = {x['Topic']: get_ollama_topic(x['Representative_Docs'], x['Representation']) for x in topic_model_patents.get_topic_info().to_dict(orient='records')}
topic_dict_patents[-1] = 'Outlier Topic'
topic_dict_patents

{-1: 'Outlier Topic',
 0: 'Solar Heat System',
 1: 'Water Conservation Hydropower',
 2: 'Wind Power Gener',
 3: 'Water Solar Plant Energi Utility Device',
 4: 'Solar Street Lamp',
 5: 'Hydrogen Energy System',
 6: 'Power Supply Modules',
 7: 'Hydropower Station Operations',
 8: 'Solar Panel Mounting',
 9: 'Solar Cell Preparation Methods',
 10: 'Monitor & Camera',
 11: 'Photovolta Board Support',
 12: 'topic: Dust removal',
 13: 'Fix Plate Connect',
 14: 'Photocatalytic Materials',
 15: 'Solar Panel Rotation and Connection',
 16: 'Insect Trap Lamps',
 17: 'Cabinet with Door and Side Parts',
 18: 'Solar Roof Design',
 19: 'Water Conservation Gate',
 20: 'Curtain wall construction'}

In [94]:
# update model
topic_model_media.set_topic_labels(topic_dict_media)
topic_model_patents.set_topic_labels(topic_dict_patents)

In [95]:
# see new topic names
topic_model_media.get_topic_info()

,Topic,Count,Name,CustomName,Representation,KeyBERT,MMR,Representative_Docs
0,-1,1685,-1_solar_energi_pv_power,Outlier Topic,"[solar, energi, pv, power, storag, project, sy...","[solar, pv, industri, panel, technolog, electr...","[solar, energi, pv, power, storag, project, sy...",[solar electr 16 2022 rel modest increas due l...
1,0,4938,0_energi_ga_geotherm_develop,Renewable Energi Developments,"[energi, ga, geotherm, develop, hydrogen, say,...","[environment, industri, fuel, sector, oil, car...","[energi, ga, geotherm, develop, hydrogen, say,...",[click allow agre store cooki devic enhanc sit...
2,1,780,1_vehicl_tesla_ev_electr,topic: Tesla ev sales,"[vehicl, tesla, ev, electr, charg, car, cleant...","[tesla, ev, vehicl, ford, car, electr, volkswa...","[vehicl, tesla, ev, electr, charg, car, cleant...",[plugin electr vehicl fulli electr vehicl well...
3,2,617,2_solar_energi_project_renew,Solar Energi Projects,"[solar, energi, project, renew, power, commun,...","[solar, sector, energi, industri, build, proje...","[solar, energi, project, renew, power, commun,...",[consum energi princip subsidiari cm energi ag...
4,3,278,3_power_grid_system_batteri,Grid-tied batteri storage,"[power, grid, system, batteri, energi, storag,...","[microgrid, solar, pv, grid, voltag, panel, el...","[power, grid, system, batteri, energi, storag,...",[follow top solar invert product 2021 take loo...
5,4,226,4_batteri_lithium_ion_materi,Lithium ion batteri,"[batteri, lithium, ion, materi, energi, techno...","[lithium, batteri, electr, ev, vehicl, manufac...","[batteri, lithium, ion, materi, energi, techno...",[low carbon energi revolut becom unstopp jugge...
6,5,182,5_solar_manufactur_modul_pv,Solar Manufacturing,"[solar, manufactur, modul, pv, us, panel, year...","[solar, industri, tariff, panel, sector, pv, m...","[solar, manufactur, modul, pv, us, panel, year...",[solar energi industri associ seia wood macken...
7,6,152,6_dn_media_group_recharg,Offshore Wind Energy,"[dn, media, group, recharg, read, privaci, dat...","[sector, industri, privaci, data, polici, farm...","[dn, media, group, recharg, read, privaci, dat...",[global offshor wind player bluefloat energi l...
8,7,119,7_cell_perovskit_effici_solar,Perovskit Solar Cell Efficiency,"[cell, perovskit, effici, solar, pv, data, mod...","[photovolta, solar, pv, cell, industri, silico...","[cell, perovskit, effici, solar, pv, data, mod...",[result confirm germani institut solar energi ...
9,8,88,8_wind_power_renew_solar,Renewable Power Growth,"[wind, power, renew, solar, energi, coal, gene...","[coal, solar, demand, pv, electr, fuel, power,...","[wind, power, renew, solar, energi, coal, gene...",[accord new data u feder energi regulatori com...


In [96]:
topic_model_patents.get_topic_info()

,Topic,Count,Name,CustomName,Representation,KeyBERT,MMR,Representative_Docs
0,-1,2214,-1_solar_energi_panel_connect,Outlier Topic,"[solar, energi, panel, connect, arrang, devic,...","[photovolta, solar, panel, electr, system, mot...","[solar, energi, panel, connect, arrang, devic,...",[util model disclos electr storag devic compri...
1,0,888,0_heat_water_energi_system,Solar Heat System,"[heat, water, energi, system, solar, temperatu...","[heater, solar, photovolta, thermal, heat, pum...","[heat, water, energi, system, solar, temperatu...",[invent disclos build energi supplement system...
2,1,701,1_water_plate_connect_hydropow,Water Conservation Hydropower,"[water, plate, connect, hydropow, fix, conserv...","[pipe, clamp, plate, flow, hydropow, drainag, ...","[water, plate, connect, hydropow, fix, conserv...",[util model relat technic field water conserv ...
3,2,441,2_wind_gener_power_blade,Wind Power Gener,"[wind, gener, power, blade, energi, rotor, rot...","[wind, rotor, vehicl, wheel, grid, speed, rota...","[wind, gener, power, blade, energi, rotor, rot...",[invent disclos doubl wind wheel wind turbin g...
4,3,315,3_water_solar_plant_energi,Water Solar Plant Energi Utility Device,"[water, solar, plant, energi, util, devic, pip...","[panel, environ, inlet, solar, system, model, ...","[water, solar, plant, energi, util, devic, pip...",[util model disclos multi theme environ friend...
5,4,303,4_lamp_light_street_solar,Solar Street Lamp,"[lamp, light, street, solar, connect, panel, r...","[lamp, solar, panel, lampshad, light, led, pol...","[lamp, light, street, solar, connect, panel, r...",[util model relat street lamp particular 5g co...
6,5,293,5_hydrogen_system_fuel_energi,Hydrogen Energy System,"[hydrogen, system, fuel, energi, storag, produ...","[hydrogen, fuel, system, inlet, tank, pipe, pr...","[hydrogen, system, fuel, energi, storag, produ...",[util model relat heat control system hydrogen...
7,6,266,6_power_modul_voltag_circuit,Power Supply Modules,"[power, modul, voltag, circuit, charg, control...","[photovolta, solar, voltag, panel, circuit, po...","[power, modul, voltag, circuit, charg, control...",[util model provid power suppli devic uninterr...
8,7,262,7_hydropow_station_data_method,Hydropower Station Operations,"[hydropow, station, data, method, water, calcu...","[hydropow, reservoir, hydroelectr, system, sta...","[hydropow, station, data, method, water, calcu...",[invent relat technic field hydropow gener pro...
9,8,190,8_solar_panel_connect_plate,Solar Panel Mounting,"[solar, panel, connect, plate, fix, frame, rod...","[panel, solar, frame, clamp, pivot, bracket, m...","[solar, panel, connect, plate, fix, frame, rod...",[util model suitabl technic field solar panel ...


## Visualise the topics

In [97]:
# TODO